# Import

In [11]:
import pandas as pd

df = pd.read_csv('fuck.csv', names = ['id_user', 'date', 'hours', 'id_item', 'price', 'qty'], skiprows=1)
dfTruth = pd.read_csv('../ressources/ground_truth.csv')

uniqueUsersTruth = dfTruth['id_user'].unique()

f = pd.DataFrame(index=uniqueUsersTruth, columns=[i + 1 for i in range(13)]).fillna(value='DEL')

# Parse

In [12]:
def parseColumns(df):
    # DATE & HOURS to datetime
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['hours'])
    df.drop('date', axis=1, inplace=True)
    df.drop('hours', axis=1, inplace=True)
    return df

parseColumns(df)
parseColumns(dfTruth)

,id_user,id_item,price,qty,datetime
0,17850,85123A,2.55,6,2010-12-01 08:26:00
1,17850,71053,3.39,6,2010-12-01 08:26:00
2,17850,84406B,2.75,8,2010-12-01 08:26:00
3,17850,84029G,3.39,6,2010-12-01 08:26:00
4,17850,84029E,3.39,6,2010-12-01 08:26:00
...,...,...,...,...,...
307049,12680,22613,0.85,12,2011-12-09 12:50:00
307050,12680,22899,2.10,6,2011-12-09 12:50:00
307051,12680,23254,4.15,4,2011-12-09 12:50:00
307052,12680,23255,4.15,4,2011-12-09 12:50:00


# Sliced

In [13]:
def sliceDFPerMonths(df):
    res = []
    
    minDate = min(df['datetime'])
    maxDate = max(df['datetime'])
        
    current = minDate
    
    while current.to_period('M') <= maxDate.to_period('M'):
        mask = (df['datetime'].dt.to_period('M') == current.to_period('M'))
        res.append(df.loc[mask])
        current += pd.DateOffset(months=1)
    return res

df = sliceDFPerMonths(df)
dfTruth = sliceDFPerMonths(dfTruth)

# Autres

In [ ]:
users = []
usersTruth = []
seuilPrice = 200
seuilQty = 200
histo = {}

for index, currentMonth in enumerate(df):
    users.append(currentMonth.groupby(['id_user']).sum())

for index, currentMonth in enumerate(dfTruth):
    usersTruth.append(currentMonth.groupby(['id_user']).sum())
    
for index in range(13):
    for i, user in users[index].iterrows():
        for j, userTruth in usersTruth[index].iterrows():
            diffPrice = abs(user['price'] - userTruth['price'])
            diffQty = abs(user['qty'] - userTruth['qty'])
            if diffPrice < seuilPrice and diffQty < seuilQty:
                if userTruth.name not in histo:
                    histo[userTruth.name] = diffPrice + diffQty
                    f.loc[userTruth.name][index + 1] = user.name
                elif histo[userTruth.name] > diffPrice + diffQty:
                    histo[userTruth.name] = diffPrice + diffQty
                    f.loc[userTruth.name][index + 1] = user.name
    print('Month ' + str(index))

Month 0
Month 1
Month 2
Month 3
Month 4
Month 5
Month 6
Month 7


# Résultats

In [68]:
for index in range(13):
    print(str(len(f[index + 1].unique()) - 1) + '/' + str(len(usersTruth[index])))

470/798
0/661
0/686
0/888
0/780
0/965
0/912
0/861
0/854
0/1176
0/1272
0/1544
0/549


In [149]:
f.to_csv("./f.csv", sep=',', index=True, header=None)

In [50]:
newDf = df.groupby(['id_user', 'date', 'qty']).size()
newDf = df.groupby(['id_user', 'date', 'qty']).size()
print(newDf)

dfTruth.loc[(dfTruth['date'] == '2011/03/15') & (dfTruth['qty'] == 9)]

id_user     date        qty
1           2           6      1
aabjhiqueh  2011/03/15  3      2
                        7      1
                        18     2
                        20     2
                              ..
zzwwquevsg  2011/03/15  18     2
                        28     1
                        30     3
                        32     1
                        270    1
Length: 122555, dtype: int64


,id_user,date,hours,id_item,price,qty
62020,13451,2011/03/15,15:37,22857,0.85,9
